<a href="https://colab.research.google.com/github/jjaw89/spring_2025_dl_audio_project/blob/main/LibriSpeechDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import librosa
import numpy as np
import os
from torch.utils.data import Dataset

%cd '/content/drive/MyDrive/DeepLearningBootcamp/LibriSpeech/dev-clean'

In [ ]:
class LibriSpeechDataset(Dataset):

  def __init__(self, path, steps = 256, num_specs = 7647):
    self.mel_specs = self.mel_specs = torch.zeros(1, 128, steps)
    self.sample_rates = torch.tensor([0])

    num_files_opened = 0

    for speaker_dir in os.listdir(path):
      speaker_path = path + "/" + speaker_dir
      for chapter_dir in os.listdir(speaker_path):
        chapter_path = speaker_path + "/" + chapter_dir
        for file in os.listdir(chapter_path):
          # checks file extension and stops when we hit desired number of spectrograms (num_specs)
          if file.endswith('.flac') and self.mel_specs.shape[0] - 1 < num_specs:

            try:
              # get audio file and convert to log mel spectrogram
              speech, rate = librosa.load(chapter_path + "/" + file, sr = 44100)
              mel_spec = torch.from_numpy(self._mel_spectrogram(speech, rate))

              # Saves the total number of 128 x (steps) spectrograms
              num_slices = mel_spec.shape[1] // steps

              # chop off the last bit so that number of stft steps is a multiple of step size
              mel_spec = mel_spec[ : , 0 : num_slices*steps]

              # reshape the tensor to have many spectrograms of size 128 x (steps)
              mel_spec = torch.transpose(torch.reshape(mel_spec, (128, num_slices, steps)), 0, 1)

              # concatenate tensor to the full tensor in the Dataset object
              self.mel_specs = torch.cat((self.mel_specs, mel_spec), 0)
              self.sample_rates = torch.cat((self.sample_rates, torch.Tensor([rate])), 0)
              num_files_opened += 1

            except:
              print("failed to open " + file)


    # chop off the zero layer we initialized with
    self.mel_specs = self.mel_specs[1:]
    self.sample_rates = self.sample_rates[1:]
    print("opened " + str(num_files_opened) + " files")
    print("collected " + str(self.mel_specs.shape[0]) + " chunks")

  def __len__(self):
    return self.mel_specs.shape[0]

  def __getitem__(self, ndx):
    return self.mel_specs[ndx], self.sample_rates[ndx]

  def _mel_spectrogram(self, audio, rate):
    # compute the log-mel-spectrogram of the audio at the given sample rate
    return librosa.power_to_db(librosa.feature.melspectrogram(y = audio, sr = rate))

In [ ]:
path = "/content/drive/MyDrive/DeepLearningBootcamp/LibriSpeech/dev-clean"
data = LibriSpeechDataset(path)